<a href="https://colab.research.google.com/github/raghavkaushik2004/AlexNet-architechture/blob/main/alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
from keras import datasets, layers, models, losses
from tensorflow import newaxis

Now, the dataset MNIST has the dimensions of 28x28, so for making the number of channels = 3, we repeat it 3 times to get 28x28x3. So we have to reshape it to 227x227x3. The paper says it is 224x224x3, but for the math to come out right(The original paper said different numbers, but Andrej Karpathy, the former head of computer vision at Tesla, said it should be 227×227×3 (he said Alex didn't describe why he put 224×224×3)). So, we reshape the image size to 227x227x3.

In [3]:
# x_train = tf.pad(x_train, [[0, 0], [98, 97], [98, 97]]) / 255
# x_train = tf.expand_dims(x_train, axis = -1)
# x_train = tf.repeat(x_train, 3, axis = -1)
# x_train = tf.image.resize(x_train, (255, 255))

# Colab crashes due to RAM constraints

In [28]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
x_train = tf.image.resize(x_train, (227, 227))
x_test = tf.image.resize(x_test, (227, 227))
x_train = x_train[... , tf.newaxis]
x_test = x_test[... , tf.newaxis]
x_train = tf.repeat(x_train, 3, axis = -1)
x_test = tf.repeat(x_test, 3, axis = -1)

In [29]:
model = models.Sequential()

Using Bilinear Interpolation to resize the image

In [30]:
model.add(layers.experimental.preprocessing.Resizing(227, 227, interpolation="bilinear", input_shape=x_train.shape[1:]))

# 227x227x3 is now converted to 55x55x96 using
model.add(layers.Conv2D(96, 11, strides = 4, padding = "same"))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))

# max pooling layer applied on 55x55x96 with 3x3 window and stride = 2 to obtain an img of 27x27x96
model.add(layers.MaxPooling2D(3, strides = 2))

# 'same' convolution with 256 5x5 filers to get img of size 27x27x256
model.add(layers.Conv2D(256, 5, strides = 4, padding = "same"))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))

# max pooling layer applied on 27x27x256 with 3x3 filter and stride = 2 to obtain an img of 13x13x256
model.add(layers.MaxPooling2D(3, strides = 2))

# 2 layers of 'same' convolution with 384 3x3 filters which makes the dim of the img to 13x13x384
model.add(layers.Conv2D(384, 3, padding = "same"))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, padding = "same"))
model.add(layers.Activation('relu'))

# one 'same' convolution with 256 3x3 filter which makes the dim of the img to 13x13x256
model.add(layers.Conv2D(256, 3, padding = "same"))
model.add(layers.Activation('relu'))

# max pooling layers with 6x6 window and stride = 2 to obtain an img of size 6x6x256
model.add(layers.MaxPooling2D(3, strides = 2))

# flattening the data into one-dimensional vector
model.add(layers.Flatten())

# first fully connected layer